In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from itertools import cycle, repeat, permutations, combinations
sns.set_theme(style="whitegrid")

In [2]:
base_dir = "/Users/scharlottej13/Nextcloud/linkedin_recruiter/model-outputs"
model_name = "cohen_eu_dist_biggest_cities_plus_recip_pairs"

## Model results -- EU+ countries

In [3]:
a_file = open(f"{base_dir}/{model_name}.txt")
lines = a_file.readlines()
for line in lines:
    print(line)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/scharlottej13/Nextcloud/linkedin_recruiter/model-outputs/cohen_eu_dist_biggest_cities_plus_recip_pairs.txt'

In [4]:
fit_df = pd.read_csv(f"{base_dir}/{model_name}.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/scharlottej13/Nextcloud/linkedin_recruiter/model-outputs/cohen_eu_dist_biggest_cities_plus_recip_pairs.csv'

In [17]:
print(f"Included countries:\n{fit_df[['country_dest']].drop_duplicates().sort_values(by='country_dest').reset_index(drop=True)}")

Included countries:
      country_dest
0          Austria
1          Belgium
2         Bulgaria
3           Cyprus
4          Denmark
5           France
6          Germany
7           Greece
8          Hungary
9          Ireland
10           Italy
11      Luxembourg
12     Netherlands
13          Poland
14        Portugal
15         Romania
16        Slovakia
17           Spain
18          Sweden
19     Switzerland
20  United Kingdom


In [19]:
log_cols = ['flow_median', 'preds', 'dist_biggest_cities', 'users_orig_median', 'users_dest_median', 'area_orig', 'area_dest']
fit_df[[f'exp_{x}' for x in log_cols]] = fit_df[[x for x in log_cols]].apply(lambda x: np.power(10, x))

In [20]:
fit_df['exp_resids'] = fit_df['exp_preds'] - fit_df['exp_flow_median']
fit_df['pct_error'] = ((fit_df['flow_median'] - fit_df['preds']) / fit_df['preds']) * 100
fit_df['resids_quant'] = pd.qcut(
    fit_df['resids'], 4,
    labels=list(range(1,5))
).astype(int)
fit_df['pct_error_quant'] = pd.qcut(
    fit_df['pct_error'], 4,
    labels=list(reversed(range(1,5)))
).astype(int)

In [45]:
def fancy_subplots(y_val='resids', num_cols=2):
    var_label_dict = {
        'preds': 'Predictions (log10)', 'dist_biggest_cities': 'Distance (log10)',
        'users_orig_median': 'Users (origin location) (log10)',
        'users_dest_median': 'Users (destination location) (log10)',
        'area_orig': 'Area (origin) (log10)', 'area_dest': 'Area (destination) (log10)',
        'csl': 'P(Common Spoken Language)'
    }
    # quickly set titles depending on residuals
    if y_val == 'resids':
        type = 'Raw'
    elif y_val == 'sresids':
        type = 'Standardized'
    num_rows = int(np.ceil(len(var_label_dict.keys())/num_cols))
    # STEP 1
    fig = make_subplots(
        rows=num_rows, cols=num_cols,
        subplot_titles=(list(var_label_dict.values())),
        shared_yaxes=True, y_title=f'{type} Residuals', vertical_spacing=0.085)
    # to iterate between columns
    colIterator = cycle(range(1, num_cols+1))
    # and over rows
    rows = sorted([x for x in range(1, num_rows+1)] * 2)
    for i, (k, v) in enumerate(var_label_dict.items()):
        plt_kwargs = {
            'x': fit_df[f'{k}'], 'y': fit_df[f'{y_val}'], 'name': v, 'mode': 'markers',
            'text': list(zip(fit_df['country_orig'], fit_df['country_dest']))
        }
        fig.add_trace(go.Scatter(**plt_kwargs),row=rows[i], col=next(colIterator))
        fig.add_hline(y=0)
    fig.update_layout(
        height=1400, width=700, title_text=f'Partial {type} Residual Plots',
        showlegend=False
    )
    fig.show()

### summary statistics: residuals

In [72]:
# Standardized Residual (i) = Residual (i) / Standard Deviation of Residuals
fit_df[['resids', 'sresids']].describe().rename(columns={'resids': 'residuals', 'sresids': 'standardized residuals'})

,residuals,standardized residuals
count,4.950000e+02,495.000000
mean,1.794300e-18,0.000467
std,3.450257e-01,1.001442
min,-9.081064e-01,-2.624070
25%,-2.531258e-01,-0.731603
50%,-1.756212e-02,-0.051045
75%,2.241776e-01,0.649455
max,9.336382e-01,2.718351


In [70]:
# fancy_subplots()

#### perhaps not the best for countries w/ small area (e.g. Luxembourg, Malta)

In [47]:
fancy_subplots(y_val='sresids')

In [63]:
fig = go.Figure()
vals = fit_df['contig'].unique()
for val in vals:
    if val == 0:
        val_name = 'No shared land border'
    else:
        val_name = 'Shared land border'
    fig.add_trace(go.Violin(
        x=fit_df['contig'][fit_df['contig'] == val],
        y=fit_df['sresids'][fit_df['contig'] == val],
        text=list(zip(
            fit_df['country_orig'][fit_df['contig'] == val],
            fit_df['country_dest'][fit_df['contig'] == val]
        )),
        name=val_name))
fig.update_layout(
        title_text=f'Partial Standardized Residual Plot<br><i>by whether two countries share a land border',
        xaxis_visible=False, xaxis_showticklabels=False
    )
fig.update_traces(
    meanline_visible=True,
    box_visible=True,
    points='all', # show all points
    jitter=0.05, # add some jitter on points for better visibility
    hoverinfo="text" # don't show values, only text labels on hover
)
fig.show()

## outliers & influential data points

In [12]:
# https://www.oreilly.com/library/view/practical-statistics-for/9781491952955/ch04.html#InfluenceExample
# https://online.stat.psu.edu/stat462/node/172/

#### standardized residuals, "number of standard errors away from regression line" (normalized so SD = 1) quantify how large the residuals are in standard deviation units. abs(std resid) > 3 is sometimes a threshold for outlier detection

#### first, very positive residuals (predictions are an underestimate, or, more people want to relocate from A to B than expected)

In [11]:
# residual = (obs - preds)
# first, very positive residuals (underestimate of y)
fit_df.loc[fit_df['sresids'].abs() > np.log10(3)].sort_values(
    by='sresids', ascending=False
)[['country_orig', 'country_dest']].drop_duplicates().head(10)

,country_orig,country_dest
248,Italy,Switzerland
162,Greece,Cyprus
166,Greece,Luxembourg
311,Poland,Iceland
116,France,Luxembourg
42,Croatia,Austria
319,Poland,Norway
89,Estonia,Finland
173,Hungary,Austria
237,Italy,Luxembourg


#### and the very negative residuals (predictions are larger than observed values)

In [13]:
# and the very negative residuals (overestimate of y)
fit_df.loc[fit_df['sresids'].abs() > np.log10(3)].sort_values(
    by='sresids')[['country_orig', 'country_dest']].drop_duplicates().head(10)

,country_orig,country_dest
431,Sweden,Romania
433,Sweden,Slovenia
432,Sweden,Slovakia
124,France,Slovenia
341,Portugal,Romania
411,Sweden,Bulgaria
287,Netherlands,Slovenia
413,Sweden,Cyprus
75,Denmark,Croatia
427,Sweden,Netherlands


In [13]:
# number of predictors
P = 7
# number of observations
n = len(fit_df)
print(f"number of predictors: {P} \nnumber of observations: {len(fit_df)}")

number of predictors: 7 
number of observations: 495


In [14]:
print("calculation of leverage statistic, based only on the independent variables")
hat_threshold = (2 * (P + 1))/n
print(f"Recommended cutoff from Hoaglin & Welsch: {hat_threshold}")
top10 = fit_df.loc[fit_df['hat_values'] > hat_threshold].sort_values(
    by='hat_values', ascending=False
)[['country_orig', 'country_dest']].drop_duplicates().reset_index(drop=True).head(10)
print(f"Top 10 country pairs that are above this threshold:\n{top10}")

calculation of leverage statistic, based only on the independent variables
Recommended cutoff from Hoaglin & Welsch: 0.03232323232323232
Top 10 country pairs that are above this threshold:
  country_orig country_dest
0        Malta       Cyprus
1      Finland      Estonia
2      Estonia      Finland
3   Luxembourg       France
4       Cyprus        Malta
5       Latvia        Malta
6   Luxembourg      Belgium
7       Sweden        Malta
8      Austria     Slovakia
9      Ireland        Malta


In [15]:
print("Cook's Distance is a combination of standardized residuals & leverage")
cooks_threshold = 4 / (n - P - 1)
print(f"threshold: {cooks_threshold}")
# "influential" country pairs
top_10 = fit_df.loc[fit_df['cooks_dist'] > cooks_threshold].sort_values(
    by='cooks_dist', ascending=False
)[['country_orig', 'country_dest']].drop_duplicates().head(10)
print(f"Top 10 country pairs over the threshold:\n{top_10}")

Cook's Distance is a combination of standardized residuals & leverage
threshold: 0.008213552361396304
Top 10 country pairs over the threshold:
    country_orig    country_dest
89       Estonia         Finland
261        Malta          Cyprus
116       France      Luxembourg
93       Finland         Estonia
248        Italy     Switzerland
22       Belgium      Luxembourg
162       Greece          Cyprus
372     Slovakia         Austria
260   Luxembourg     Switzerland
436       Sweden  United Kingdom


In [68]:
fit_df['size'] = np.sqrt(fit_df['cooks_dist']*10)

In [69]:
# using standard residuals, cook's distance, & hat values
fig = px.scatter(
    fit_df, x='hat_values', y='sresids', size='size',
    hover_data=['country_orig', 'country_dest'],
    labels = {'hat_values': 'hat values', 'sresids': 'std. residuals'})
fig.add_hline(y=3, line_dash='dash')
fig.add_hline(y=-3, line_dash='dash')
fig.add_vline(x=hat_threshold, line_dash='dash')
fig.show()

NameError: name 'hat_threshold' is not defined